In [1]:
from load_real_graphs import (get_available_datasets, get_network_from_konect, CHOOSEN_DATASETS)
from generate_graph import recreate_graph

from graph import get_graph_measurements, compare_graph_measurements, print_comparison_results
from recreate_graph import graph_to_dataframe, get_trained_model, recreate_by_priority_rank


Using TensorFlow backend.
/home/smoq/.virtualenvs/cna/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dataset_name = 'moreno_seventh'
graph = get_network_from_konect(dataset_name)

skipping moreno_seventh - already exists
skipping moreno_seventh - already exists
Loading graph...
Attaching attributes (graph measurements)...
Attaching real attributes...


In [3]:
graph.nodes[0]

{'betweenness_centrality': 0.03237684388631609,
 'closeness_centrality': 0.9655172413793104,
 'degree_centrality': 1.0357142857142856,
 'gender': 'male',
 'pagerank': 0.048327152998040294}

In [4]:
df = graph_to_dataframe(graph)

In [5]:
model = get_trained_model(df)

Epoch 1/128
841/841 [==============================] - 0s 76us/step - loss: 0.7331
Epoch 2/128
841/841 [==============================] - 0s 16us/step - loss: 0.7139
Epoch 3/128
841/841 [==============================] - 0s 18us/step - loss: 0.7002
Epoch 4/128
841/841 [==============================] - 0s 20us/step - loss: 0.6906
Epoch 5/128
841/841 [==============================] - 0s 13us/step - loss: 0.6830
Epoch 6/128
841/841 [==============================] - 0s 14us/step - loss: 0.6770
Epoch 7/128
841/841 [==============================] - 0s 16us/step - loss: 0.6721
Epoch 8/128
841/841 [==============================] - 0s 27us/step - loss: 0.6678
Epoch 9/128
841/841 [==============================] - 0s 18us/step - loss: 0.6633
Epoch 10/128
841/841 [==============================] - 0s 18us/step - loss: 0.6592
Epoch 11/128
841/841 [==============================] - 0s 15us/step - loss: 0.6550
Epoch 12/128
841/841 [==============================] - 0s 17us/step - loss: 0.6509
E

841/841 [==============================] - 0s 15us/step - loss: 0.4618
Epoch 99/128
841/841 [==============================] - 0s 17us/step - loss: 0.4603
Epoch 100/128
841/841 [==============================] - 0s 14us/step - loss: 0.4589
Epoch 101/128
841/841 [==============================] - 0s 29us/step - loss: 0.4576
Epoch 102/128
841/841 [==============================] - 0s 20us/step - loss: 0.4562
Epoch 103/128
841/841 [==============================] - 0s 14us/step - loss: 0.4549
Epoch 104/128
841/841 [==============================] - 0s 14us/step - loss: 0.4535
Epoch 105/128
841/841 [==============================] - 0s 12us/step - loss: 0.4521
Epoch 106/128
841/841 [==============================] - 0s 14us/step - loss: 0.4509
Epoch 107/128
841/841 [==============================] - 0s 19us/step - loss: 0.4496
Epoch 108/128
841/841 [==============================] - 0s 16us/step - loss: 0.4485
Epoch 109/128
841/841 [==============================] - 0s 15us/step - loss: 0.

In [6]:
new_graph = recreate_by_priority_rank(graph, df, model)

In [7]:
graph_measurements = get_graph_measurements(graph)
new_graph_measurements = get_graph_measurements(new_graph)

comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)

In [8]:
print_comparison_results(comparison)

(KS test p-value)  degree_centrality: 0.009388569191              [passed: False]
(KS test p-value)  closeness_centrality: 0.003701331986           [passed: False]
(KS test p-value)  betweenness_centrality: 0.098002397010         [passed: True]
(KS test p-value)  pagerank: 0.184429542118                       [passed: True]
(abs distance)     average_shortest_path_length: 0.084112149533   [passed: False]
(abs distance)     diameter: 0.000000000000                       [passed: True]
(abs distance)     degree_centralization: 0.407526881720          [passed: False]
(abs distance)     density: 0.135714285714                        [passed: False]


In [9]:
graph.number_of_edges()

280

In [10]:
new_graph.number_of_edges()

242